### Note

0. 要先去 verdict-cls-debug/uie 訓練好一個 uie 模型，把 checkpoint 放到 uie_model 內，step3再指定他才能跑，詳見verdict-cls-debug/uie/pipeline.ipynb（ckp 太大丟不到github）
1. 進入點：verdict-cls-debug/utc/zero_shot_text_classification
2. 很多檔案感覺都不需要，ex metric.py, modeling.py, utc_trainer.py (很多都是為了實驗存在，真正需要的檔案應該可以收斂很多)
3. verdict-cls-debug/utc/dev 這個資料夾主要是開發時候實驗的一些拉ㄐpy檔案

##### Step 1
merge all .json data in labelstudio_data/

In [ ]:
python read_and_merge_data.py   

##### Step 2
split merged data (read_and_merge_data.py output) into data/(train, test, dev)

In [ ]:
python label_studio.py  --label_studio_file ./labelstudio_data/formal_data/classification.json --options ./labelstudio_data/label.txt

##### Step 3
用 UIE 截短長文本 （依序處理 .data/train.txt, .data/dev.txt, .data/test.txt）

感覺可以更 clean code:
- uie_preprocessin.py 
- .uie_model/filter_text.py

In [ ]:
python uie_preprocessing.py --max_seq_len 768 --threshold 0.0 --uie_model_name_or_path uie_model/checkpoint-2790/ --out_folder_name processed_data_1000

##### Step 4
Training

note: 最大長度跟 step 3 一樣比較合理

In [ ]:
python run_train.py  \
    --device gpu \
    --logging_steps 5 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 1000 \
    --model_name_or_path utc-base \
    --output_dir ./checkpoint/model_best/data_1000 \
    --dataset_path ./data/processed_data_1000/data_1000 \
    --max_seq_length 768  \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --num_train_epochs 25 \
    --learning_rate 1e-4 \
    --do_train \
    --do_eval \
    --do_predict \
    --do_export \
    --export_model_dir ./checkpoint/model_best/data_1000 \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model macro_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1 \
    --save_plm 

##### 跑 Eval

In [ ]:
python run_eval.py --test_path ./data/processed_data_1000/data_1000/test.txt --max_seq_len 768 --per_device_eval_batch_size 8  --model_path ./checkpoint/model_best/data_1000 --output_dir ./checkpoint_final

Get all data

python label_studio.py --save_dir ./all_data --splits 1 0 0 --options labelstudio_data/label.txt


### Run baseline utc

In [ ]:
python baseline_train.py \
    --device gpu \
    --logging_steps 5 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 1000 \
    --model_name_or_path utc-base \
    --output_dir ./checkpoint/model_best/baseline \
    --dataset_path ./data/data_1000 \
    --max_seq_length 768  \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --num_train_epochs 25 \
    --learning_rate 1e-4 \
    --do_train \
    --do_eval \
    --do_predict \
    --do_export \
    --export_model_dir ./checkpoint/model_best/baseline \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model macro_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1 \
    --save_plm 